In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from matplotlib import pyplot as plt

import os
from tensorflow import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Embedding, GRU, Bidirectional, Concatenate, Dropout, Layer, Add, LayerNormalization
from keras.utils import pad_sequences
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk import edit_distance

C:\Users\Smeki RedTech\anaconda3\envs\tensorflowgpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


U fajlu funkcije.py smo implementirali mnoge funkcije za pretprocesiranje i neke druge korisne stvari koje cemo da koristimo u vise notebooka.

In [3]:
from funkcije import *

Kako smo imali problem sa limitacijom memorije graficke karte, koristimo poseban setting koji moze da pomnogne u tom slucaju.

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

Inicijalizujemo callback koji cemo da koristimo pri svakom treniranju modela. Naime, svaki model treniramo do trenutka dok performanse na validacionom skupu ne opadne.

In [5]:
early_stopping = EarlyStopping(patience = 10, restore_best_weights = True, monitor = 'val_loss', mode = 'min', verbose = 1)

In [6]:
embedding_size = 300

Pravimo encoder-decoder model za prevodioca, zasnovan na GRU celijama. Koristimo 3 GRU sloja u encoderu, gde je poslednji dvosmerni. Izmedju svaka 2 sloja imamo rezidualne konekcije na ulaze prethodnog. Decoder ima 2 GRU sloja i jedan gusti softmax sloj za klasifikaciju, pre drugog sloja takodje postoji rezidualna veza na ulaze prvog.

Latentna dimenzija je ista u svim slojevima encodera, i tretira se kao hiperparametar za optimizaciju. Posto je poslednji sloj encodera dvosmerni, latentna dimenzija decodera je duplo veca.

Koristimo pretrenirane glove embeddinge i za ulaz encodera i decodera, ali ostavljamo mogucnost da se dotreniraju.
Dropout koristimo pred svaki GRU sloj, kao i pred poslednji gusti sloj. Verovatnoca dropouta je na svakom mestu ista, i tretira se kao hiperparametar.

U decoderu pre Embedding sloja koristimo CustomDropout sloj, koji smo mi implementirali. Taj sloj nasumicno input zamenjuje sa odabranom vrednoscu, u nasem slucaju, <Unknown> tokenom, koji se kodira sa jedinicom. Nikad ne zamenjuje praznine, i za razliku od klasicnog Dropout sloja, ne reskalira vrednosti.

In [7]:
class GRU_Translation_Model(Model):
    def __init__(self, num_input_words, num_target_words, input_embedding_matrix, target_embedding_matrix, latent_dim = 256, dropout_rate = 0.5, custom_dropout_rate = 0.05):
        super(GRU_Translation_Model, self).__init__()
        
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        self.custom_dropout_rate = custom_dropout_rate
        self.num_input_words = num_input_words
        self.num_target_words = num_target_words
        self.input_embedding_matrix = input_embedding_matrix
        self.target_embedding_matrix = target_embedding_matrix
        self.embedding_size = 300
        self.input_pad_len = 80
        self.target_pad_len = 60
        
        encoder_input_tensor = Input(shape = (self.input_pad_len, ))
        modified_input = CustomDropout(1.0, custom_dropout_rate)(encoder_input_tensor)

        encoder_embedding_layer = Embedding(input_dim = num_input_words + 1, output_dim = self.embedding_size, mask_zero = True, weights = [input_embedding_matrix], trainable = True)
        encoder_embedding = encoder_embedding_layer(modified_input)
        outputs = GRU(units = latent_dim, return_sequences = True, dropout = dropout_rate)(encoder_embedding)
        outputs = Dense(units = self.embedding_size, activation = 'relu')(outputs)
        outputs = LayerNormalization()(outputs)
        next_inputs = Add()([encoder_embedding, outputs])
        outputs = GRU(units = latent_dim, return_sequences = True, dropout = dropout_rate)(next_inputs)
        outputs = Dense(units = self.embedding_size, activation = 'relu')(outputs)
        outputs = LayerNormalization()(outputs)
        main_inputs = Add()([next_inputs, outputs])
        main_inputs = LayerNormalization()(main_inputs)
        _, forward_state, backward_state = Bidirectional(GRU(units = latent_dim, return_state = True, dropout = dropout_rate))(main_inputs)
        state_h = Concatenate(axis=-1)([forward_state, backward_state])

        self.encoder = Model(encoder_input_tensor, state_h)
        
        decoder_input_tensor = Input(shape = (None, ))
        decoder_starting_state = Input(shape = (latent_dim*2,))
        decoder_embedding_layer = Embedding(input_dim = num_target_words + 1, output_dim = self.embedding_size, mask_zero = True, weights = [target_embedding_matrix], trainable = True)
        decoder_embedding = decoder_embedding_layer(decoder_input_tensor)
        decoder_outputs = GRU(units = latent_dim*2, return_sequences = True, return_state = False, dropout = dropout_rate)(decoder_embedding, initial_state = decoder_starting_state)
        decoder_outputs = Dense(units = self.embedding_size, activation = 'relu')(decoder_outputs)
        main_inputs = Add()([decoder_embedding, decoder_outputs])
        main_inputs = LayerNormalization()(main_inputs)
        decoder_outputs, decoder_state = GRU(units = latent_dim*2, return_sequences = True, return_state = True, dropout = dropout_rate)(main_inputs, initial_state = decoder_starting_state) 
        output = Dropout(dropout_rate)(decoder_outputs)
        output = Dense(units = num_target_words + 1, activation = 'softmax')(output)
        
        self.decoder = Model([decoder_input_tensor, decoder_starting_state], [output, decoder_state])
        
    def call(self, x):
        encoder_output = self.encoder(x[0])
        decoder_output,_ = self.decoder([x[1],encoder_output])
        return decoder_output
    

    def translate(self, encoder_input, decoder_input):
        decoder_state = self.encoder(encoder_input)
        data_size = encoder_input.shape[0]
        decoder_output = np.zeros((data_size, self.target_pad_len - 1))
        for i in range(self.target_pad_len - 1):
            decoder_output_temp, decoder_state = self.decoder.predict([decoder_input, decoder_state], verbose = 0, batch_size = 128)
            next_words = np.argmax(decoder_output_temp, axis = -1)
            decoder_input = next_words
            decoder_output[:, i] = next_words.reshape((data_size,))
        return decoder_output

Funkcija train_and_evaluate istrenira model na train_data i evaluira ga na val_data. Kao mere za evaluaciju pamti Word Error Rate, BLEU 1-4 (smoothovan). Smoothovan BLEU se pokazao kao bolji u slucajevima kada recenicu imaju mali broj reci, sto je slucaj sa glossovanim recenicama.

Takodje cuva dostignut val_loss, kao i broj epoha do konvergencije.

Funkcija takodje daje mogucnost treniranja Embedding slojeva sa razlicitim learning_rate-om od ostalih slojeva. Ideja iza ovoga je da, posto su pocetne vrednosti Embedding slojeva pretrenirane, ima smisla da se taj sloj sporije trenira od ostalih.

In [8]:
#Trenira model na train_data i evaluira ga na val_data
#Embedding learning rate je poseban learning_rate koji se koristi u embedding slojevima, iz razloga sto oni vec imaju pretrenirane podatke za pocetne vrednosti
#Model se trenira dok val_loss ne krene da raste, i cuva tezine epohe koja ima najbolji val_loss
#Koristi se u cv_evaluate, napravio funkciju jer inace dolazi do prekoracenje GPU RAMa, neko je napisao da je do unakrsne validacije
def train_and_evaluate(train_data, val_data, epochs = 200, batch_size = 128, learning_rate = 0.001, latent_dim = 256, dropout_rate = 0.5, embedding_learning_rate = 0.001):
     
     input_texts, target_texts = clean_texts(train_data.iloc[:,1], train_data.iloc[:,0])
     input_word_index, target_word_index, max_input_seq_len, max_target_seq_len = analyse_texts(input_texts, target_texts)
     input_pad_len = 80
     target_pad_len = 60
     num_input_words = len(input_word_index) - 1
     num_target_words = len(target_word_index) - 1
     inverted_input_word_index = {value: key for key,value in input_word_index.items()}
     inverted_target_word_index = {value: key for (key,value) in target_word_index.items()}
     input_embedding_matrix, target_embedding_matrix = load_embedding_data_get_matrices(inverted_input_word_index, inverted_target_word_index)
     encoder_input_data, decoder_input_data, decoder_output_data = create_model_data(input_texts, target_texts, input_word_index, target_word_index, input_pad_len, target_pad_len)
     
     input_texts_val, target_texts_val = clean_texts(val_data.iloc[:,1], val_data.iloc[:,0])
     encoder_input_data_val, decoder_input_data_val, decoder_output_data_val = create_model_data(input_texts_val, target_texts_val, input_word_index, target_word_index, input_pad_len, target_pad_len)
     
     model_gru = GRU_Translation_Model(num_input_words, num_target_words, input_embedding_matrix, target_embedding_matrix, latent_dim = latent_dim, dropout_rate = dropout_rate)
     other_layers = model_gru.layers[0].layers + model_gru.layers[1].layers #Mora da se prilagodi za transformer
     embedding_layers = [other_layers.pop(2), other_layers.pop(-9)] #Paznja! Mora se prilagoditi svaki put kad se model menja

     optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers = [(Adam(learning_rate), other_layers), (Adam(embedding_learning_rate), embedding_layers)])
     model_gru.compile(optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])
     history = model_gru.fit([encoder_input_data, decoder_input_data], decoder_output_data, validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_output_data_val), epochs = epochs, batch_size = batch_size, callbacks = [early_stopping], verbose = 1)
     best_epoch = np.argmin(history.history['val_loss']) + 1
     
     best_loss = np.min(history.history['val_loss'])
     
     wer, smooth_bleu4, smooth_bleu3, smooth_bleu2, smooth_bleu1 = evaluate(model_gru, input_texts_val, target_texts_val, input_word_index, target_word_index, inverted_target_word_index, input_pad_len, target_pad_len)
     return best_epoch, best_loss, wer, smooth_bleu4, smooth_bleu3, smooth_bleu2, smooth_bleu1
 

Funkcija cv_evaluate trenira po model za svaki fold, racuna odgovarajuce metrike.

Funkcija vraca podatke iz svake instance modela, odnosno folda, da bi se dalje procesirale

In [9]:
def cv_evaluate(train_val_data = None, df_folds = None, folds = 5, epochs = 200, batch_size = 128, learning_rate = 0.001, latent_dim = 256, dropout_rate = 0.5, embedding_learning_rate = None):
    if embedding_learning_rate == None:
        embedding_learning_rate = learning_rate
    if df_folds == None:
        df_np = train_val_data.to_numpy()
        np.random.shuffle(df_np)
        total_size = df_np.shape[0]
        fold_size = total_size/folds
        df_folds = [df_np[int(i*fold_size):int((i+1)*fold_size),] for i in range(folds)]
    losses = []
    best_epochs = []
    wers = []
    smooth_bleu1s = []
    smooth_bleu2s = []
    smooth_bleu3s = []
    smooth_bleu4s = []
    for i in range(folds):
        train_folds = [fold for j, fold in enumerate(df_folds) if j!=i]
        train_folds_pd = [pd.DataFrame(data = fold) for fold in train_folds]
        train_data = pd.concat(train_folds_pd)
        val_data = pd.DataFrame(df_folds[i])
        print('Current Latent Dim:', latent_dim)
        print('Current Dropout Rate: ', dropout_rate)
        print('Current Fold: {}/{}'.format(i+1, folds))
        print('Current Learning Rate: ', learning_rate)
        print('Current Learning Rate Multiplier: ', embedding_learning_rate/learning_rate)
        
        best_epoch, best_loss, wer, smooth_bleu4, smooth_bleu3, smooth_bleu2, smooth_bleu1 = train_and_evaluate(train_data, val_data, epochs = epochs, batch_size = batch_size, learning_rate = learning_rate, latent_dim = latent_dim, dropout_rate = dropout_rate, embedding_learning_rate = embedding_learning_rate)
        best_epochs.append(best_epoch)
        losses.append(best_loss)
        wers.append(wer)
        smooth_bleu4s.append(smooth_bleu4)
        smooth_bleu3s.append(smooth_bleu3)
        smooth_bleu2s.append(smooth_bleu2)
        smooth_bleu1s.append(smooth_bleu1)
    return best_epochs, losses, wers, smooth_bleu4s, smooth_bleu3s, smooth_bleu2s, smooth_bleu1s

Funkcija grid_search_cv evaluira model za razlicite vrednosti parametara latent_dim i dropout_rate. Rezultate vraca u obliku dictionary-ja kojim je moguce pristupiti svim izracunatim metrikama.

In [10]:
def cv_grid_search(df, dropout_rates, latent_dims, epochs = 200, learning_rate = 0.0002, folds = 5):
    df_np = df.to_numpy()
    np.random.shuffle(df_np)
    total_size = df_np.shape[0]
    fold_size = total_size/folds
    df_folds = [df_np[int(i*fold_size):int((i+1)*fold_size),] for i in range(folds)]
    
    loss_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    epoch_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    wer_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    smooth_bleu4_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    smooth_bleu3_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    smooth_bleu2_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    smooth_bleu1_matrix = np.zeros((len(latent_dims),len(dropout_rates), folds))
    for i in range(len(latent_dims)):
        for j in range(len(dropout_rates)):
            best_epochs, losses, wers, smooth_bleu4s, smooth_bleu3s, smooth_bleu2s, smooth_bleu1s = cv_evaluate(df_folds = df_folds, folds = folds, epochs = epochs, learning_rate = learning_rate, latent_dim = latent_dims[i], dropout_rate = dropout_rates[j])
            print(losses)
            print(best_epochs)
            loss_matrix[i,j,:] = losses
            epoch_matrix[i,j,:] = best_epochs
            wer_matrix[i,j,:] = wers
            smooth_bleu4_matrix[i,j,:] = smooth_bleu4s
            smooth_bleu3_matrix[i,j,:] = smooth_bleu3s
            smooth_bleu2_matrix[i,j,:] = smooth_bleu2s
            smooth_bleu1_matrix[i,j,:] = smooth_bleu1s
    #Pakuju se rezultati u dictionary radi intuitivnijeg poziva funkcije
    metrics_dict = {'loss': loss_matrix, 'epoch': epoch_matrix, 'wer': wer_matrix, 'smooth_bleu4': smooth_bleu4_matrix, 'smooth_bleu3': smooth_bleu3_matrix, 'smooth_bleu2': smooth_bleu2_matrix, 'smooth_bleu1': smooth_bleu1_matrix }
    return metrics_dict

Za pocetak samo ucitavamo podatke i spajamo train i validacioni skup u jedan koji cemo koristiti za unakrsnu validaciju. Kasnije cemo model evaluirati na test skupu. Ne radimo ugnjezdenu unakrsnu validaciju jer bi predugo trajalo.

In [11]:
df_train = pd.read_csv('data/PHOENIX-2014-T.train.corpus.csv', sep='|')
df_train = df_train.drop(columns=['name','video','start','end','speaker'])
train_size = df_train.shape[0]

df_val = pd.read_csv('data/PHOENIX-2014-T.dev.corpus.csv', sep = '|')
df_val.drop(columns = ['name', 'video', 'start', 'end', 'speaker'], inplace = True)
val_size = df_val.shape[0]

df_test = pd.read_csv('data/PHOENIX-2014-T.test.corpus.csv', sep = '|')
df_test.drop(columns = ['name', 'video', 'start', 'end', 'speaker'], inplace = True)
test_size = df_test.shape[0]

df_train_val = pd.concat([df_train, df_val])
df_full = pd.concat([df_train_val, df_test])

Hiperparametri koje cemo da optimizujemo su dropout_rate i latent_dim. Learning rate fiksiramo na vrednost koja nam je iz testiranja dobro radila.

In [12]:
dropout_rates = [0.5, 0.6, 0.7, 0.8, 0.9]
latent_dims = [256, 512, 1024] 
learning_rate = 0.0002
folds = 10

Radimo grid search sa unakrsnom validacijom, metrika koju optimizujemo je smooth BLEU4

In [ ]:
metrics = cv_grid_search(df_train_val, dropout_rates, latent_dims, epochs = 200, learning_rate = learning_rate, folds = folds)
average_bleu4 = np.mean(metrics['smooth_bleu4'], axis = -1)

Current Latent Dim: 256
Current Dropout Rate:  0.5
Current Fold: 1/10
Current Learning Rate:  0.0002
Current Learning Rate Multiplier:  1.0
Epoch 1/200
54/54 [==============================] - 25s 176ms/step - loss: 0.8809 - acc: 0.1594 - val_loss: 0.7286 - val_acc: 0.2336
Epoch 2/200
54/54 [==============================] - 3s 47ms/step - loss: 0.7322 - acc: 0.2366 - val_loss: 0.6670 - val_acc: 0.2775
Epoch 3/200
54/54 [==============================] - 2s 44ms/step - loss: 0.6792 - acc: 0.2768 - val_loss: 0.6250 - val_acc: 0.3089
Epoch 4/200
54/54 [==============================] - 2s 44ms/step - loss: 0.6389 - acc: 0.3015 - val_loss: 0.5941 - val_acc: 0.3204
Epoch 5/200
54/54 [==============================] - 2s 44ms/step - loss: 0.6098 - acc: 0.3183 - val_loss: 0.5765 - val_acc: 0.3330
Epoch 6/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5885 - acc: 0.3293 - val_loss: 0.5598 - val_acc: 0.3409
Epoch 7/200
54/54 [==============================] - 2s 44ms/step 

54/54 [==============================] - 2s 44ms/step - loss: 0.3529 - acc: 0.5101 - val_loss: 0.4250 - val_acc: 0.4759
Epoch 62/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3516 - acc: 0.5096 - val_loss: 0.4228 - val_acc: 0.4758
Epoch 63/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3494 - acc: 0.5110 - val_loss: 0.4223 - val_acc: 0.4772
Epoch 64/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3478 - acc: 0.5119 - val_loss: 0.4239 - val_acc: 0.4768
Epoch 65/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3443 - acc: 0.5147 - val_loss: 0.4236 - val_acc: 0.4766
Epoch 66/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3432 - acc: 0.5165 - val_loss: 0.4237 - val_acc: 0.4766
Epoch 67/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3415 - acc: 0.5189 - val_loss: 0.4242 - val_acc: 0.4767
Epoch 68/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3394 

54/54 [==============================] - 2s 44ms/step - loss: 0.3858 - acc: 0.4784 - val_loss: 0.4211 - val_acc: 0.4726
Epoch 48/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3840 - acc: 0.4796 - val_loss: 0.4207 - val_acc: 0.4742
Epoch 49/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3819 - acc: 0.4801 - val_loss: 0.4185 - val_acc: 0.4733
Epoch 50/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3785 - acc: 0.4841 - val_loss: 0.4186 - val_acc: 0.4763
Epoch 51/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3775 - acc: 0.4855 - val_loss: 0.4177 - val_acc: 0.4768
Epoch 52/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3744 - acc: 0.4895 - val_loss: 0.4177 - val_acc: 0.4771
Epoch 53/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3724 - acc: 0.4906 - val_loss: 0.4164 - val_acc: 0.4791
Epoch 54/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3690 

54/54 [==============================] - 2s 44ms/step - loss: 0.4505 - acc: 0.4224 - val_loss: 0.4374 - val_acc: 0.4391
Epoch 27/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4468 - acc: 0.4278 - val_loss: 0.4380 - val_acc: 0.4343
Epoch 28/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4432 - acc: 0.4289 - val_loss: 0.4339 - val_acc: 0.4382
Epoch 29/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4397 - acc: 0.4335 - val_loss: 0.4327 - val_acc: 0.4423
Epoch 30/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4351 - acc: 0.4373 - val_loss: 0.4335 - val_acc: 0.4454
Epoch 31/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4326 - acc: 0.4400 - val_loss: 0.4316 - val_acc: 0.4453
Epoch 32/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4289 - acc: 0.4429 - val_loss: 0.4315 - val_acc: 0.4466
Epoch 33/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4265 

54/54 [==============================] - 2s 44ms/step - loss: 0.5402 - acc: 0.3564 - val_loss: 0.5075 - val_acc: 0.3815
Epoch 11/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5328 - acc: 0.3610 - val_loss: 0.5022 - val_acc: 0.3867
Epoch 12/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5235 - acc: 0.3653 - val_loss: 0.4946 - val_acc: 0.3913
Epoch 13/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5165 - acc: 0.3711 - val_loss: 0.4921 - val_acc: 0.3935
Epoch 14/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5093 - acc: 0.3774 - val_loss: 0.4834 - val_acc: 0.4011
Epoch 15/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5035 - acc: 0.3811 - val_loss: 0.4802 - val_acc: 0.4031
Epoch 16/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4965 - acc: 0.3873 - val_loss: 0.4767 - val_acc: 0.4086
Epoch 17/200
54/54 [==============================] - 2s 44ms/step - loss: 0.4917 

...

54/54 [==============================] - 3s 47ms/step - loss: 0.3604 - acc: 0.4988 - val_loss: 0.4233 - val_acc: 0.4774
Epoch 150: early stopping
Current Latent Dim: 256
Current Dropout Rate:  0.7
Current Fold: 2/10
Current Learning Rate:  0.0002
Current Learning Rate Multiplier:  1.0
Epoch 1/200
54/54 [==============================] - 24s 182ms/step - loss: 0.9701 - acc: 0.1100 - val_loss: 0.7972 - val_acc: 0.1992
Epoch 2/200
54/54 [==============================] - 2s 45ms/step - loss: 0.8058 - acc: 0.1868 - val_loss: 0.7382 - val_acc: 0.2265
Epoch 3/200
54/54 [==============================] - 2s 45ms/step - loss: 0.7626 - acc: 0.2152 - val_loss: 0.7041 - val_acc: 0.2463
Epoch 4/200
54/54 [==============================] - 2s 45ms/step - loss: 0.7349 - acc: 0.2337 - val_loss: 0.6765 - val_acc: 0.2758
Epoch 5/200
54/54 [==============================] - 2s 45ms/step - loss: 0.7110 - acc: 0.2503 - val_loss: 0.6507 - val_acc: 0.2953
Epoch 6/200
54/54 [==============================] -

54/54 [==============================] - 2s 45ms/step - loss: 0.4625 - acc: 0.4055 - val_loss: 0.4485 - val_acc: 0.4322
Epoch 60/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4604 - acc: 0.4078 - val_loss: 0.4475 - val_acc: 0.4326
Epoch 61/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4588 - acc: 0.4086 - val_loss: 0.4485 - val_acc: 0.4326
Epoch 62/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4573 - acc: 0.4096 - val_loss: 0.4462 - val_acc: 0.4358
Epoch 63/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4556 - acc: 0.4120 - val_loss: 0.4460 - val_acc: 0.4351
Epoch 64/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4537 - acc: 0.4148 - val_loss: 0.4446 - val_acc: 0.4376
Epoch 65/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4530 - acc: 0.4133 - val_loss: 0.4431 - val_acc: 0.4410
Epoch 66/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4522 

54/54 [==============================] - 2s 45ms/step - loss: 0.3890 - acc: 0.4699 - val_loss: 0.4164 - val_acc: 0.4735
Epoch 121/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3877 - acc: 0.4722 - val_loss: 0.4168 - val_acc: 0.4745
Epoch 122/200
54/54 [==============================] - 2s 45ms/step - loss: 0.3864 - acc: 0.4741 - val_loss: 0.4150 - val_acc: 0.4728
Epoch 123/200
54/54 [==============================] - 2s 44ms/step - loss: 0.3855 - acc: 0.4747 - val_loss: 0.4165 - val_acc: 0.4769
Epoch 124/200
54/54 [==============================] - 2s 45ms/step - loss: 0.3845 - acc: 0.4757 - val_loss: 0.4165 - val_acc: 0.4758
Epoch 125/200
54/54 [==============================] - 2s 45ms/step - loss: 0.3845 - acc: 0.4759 - val_loss: 0.4150 - val_acc: 0.4753
Epoch 126/200
54/54 [==============================] - 2s 45ms/step - loss: 0.3828 - acc: 0.4751 - val_loss: 0.4156 - val_acc: 0.4764
Epoch 127/200
54/54 [==============================] - 2s 45ms/step - loss: 

54/54 [==============================] - 2s 45ms/step - loss: 0.5331 - acc: 0.3566 - val_loss: 0.4880 - val_acc: 0.3812
Epoch 28/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5293 - acc: 0.3593 - val_loss: 0.4868 - val_acc: 0.3861
Epoch 29/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5260 - acc: 0.3617 - val_loss: 0.4856 - val_acc: 0.3852
Epoch 30/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5234 - acc: 0.3605 - val_loss: 0.4824 - val_acc: 0.3862
Epoch 31/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5217 - acc: 0.3639 - val_loss: 0.4802 - val_acc: 0.3887
Epoch 32/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5176 - acc: 0.3657 - val_loss: 0.4788 - val_acc: 0.3913
Epoch 33/200
54/54 [==============================] - 2s 45ms/step - loss: 0.5145 - acc: 0.3686 - val_loss: 0.4748 - val_acc: 0.3942
Epoch 34/200
54/54 [==============================] - 2s 44ms/step - loss: 0.5125 

Epoch 89/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4195 - acc: 0.4440 - val_loss: 0.4214 - val_acc: 0.4519
Epoch 90/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4189 - acc: 0.4428 - val_loss: 0.4227 - val_acc: 0.4502
Epoch 91/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4177 - acc: 0.4448 - val_loss: 0.4194 - val_acc: 0.4515
Epoch 92/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4154 - acc: 0.4466 - val_loss: 0.4202 - val_acc: 0.4503
Epoch 93/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4152 - acc: 0.4474 - val_loss: 0.4199 - val_acc: 0.4528
Epoch 94/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4141 - acc: 0.4487 - val_loss: 0.4193 - val_acc: 0.4534
Epoch 95/200
54/54 [==============================] - 2s 45ms/step - loss: 0.4128 - acc: 0.4498 - val_loss: 0.4187 - val_acc: 0.4546
Epoch 96/200
54/54 [==============================] - 2s 45ms/step - 

In [ ]:
plt.title('Average smooth BLEU4, crossvalidated')
plt.xlabel('Dropout Rate')
plt.xticks(range(len(dropout_rates)), dropout_rates)
plt.ylabel('Latent Dim')
plt.yticks(range(len(latent_dims)),latent_dims)
plt.imshow(average_bleu4)
plt.colorbar()
plt.show()

In [ ]:
best_config_index = np.unravel_index(np.argmax(average_bleu4), average_bleu4.shape)
best_latent_dim = latent_dims[best_config_index[0]]
best_dropout_rate = dropout_rates[best_config_index[1]]
print("Best latent dimension: ", best_latent_dim)
print("Best dropout rate: ", best_dropout_rate)

Nakon sto smo odredili optimalnu vrednost za dropout_rate i latent_dim, takodje unkarsnom validacijom optimizujemo vrednost posebnog learning rate-a za Embedding slojeve. Learning_rate_multiplier predstavlja odnos izmedju learning rate-a Embedding slojeva i learning_rate-a ostalih slojeva.

In [ ]:
learning_rate_multipliers = [0.03, 0.1, 0.3, 1, 3, 10, 30]
best_epoch_array = []
bleu4_array = []
for i in range(len(learning_rate_multipliers)):
    df_np = df_train_val.to_numpy()
    np.random.shuffle(df_np)
    total_size = df_np.shape[0]
    fold_size = total_size/folds
    df_folds = [df_np[int(i*fold_size):int((i+1)*fold_size),] for i in range(folds)]
    best_epochs, _, _, smooth_bleu4s, _, _, _ = cv_evaluate(df_train_val, df_folds, folds = folds, epochs = 200, learning_rate = learning_rate, embedding_learning_rate = learning_rate_multipliers[i]*learning_rate, latent_dim = best_latent_dim, dropout_rate = best_dropout_rate)
    best_epoch_array.append(best_epochs)
    bleu4_array.append(smooth_bleu4s)

In [ ]:
bleu4_array = np.array(bleu4_array)
best_epoch_array = np.array(best_epoch_array)
best_multiplier_index = np.argmax(np.mean(bleu4_array, -1))
bleu4_avg = np.mean(bleu4_array[best_multiplier_index])
print("Average BLEU4(smooth) on validation: ", bleu4_avg)
best_multiplier = learning_rate_multipliers[best_multiplier_index]
print("Best Multiplier for Embedding Learning Rates:", best_multiplier)
best_multiplier_epochs = best_epoch_array[best_multiplier_index]
print('Epochs to convergence on all folds: ', best_multiplier_epochs) 
epoch_avg = np.mean(best_multiplier_epochs)

Iznenadjujuce dobijamo visoku vrednost za learning_rate_multiplier, kada smo ocekivali vrednost manju od 1.

In [ ]:
plt.title('Average smooth BLEU4 for different embedding learning rates')
plt.xlabel('Learning Rate Multiplier')
plt.yticks([])
plt.xticks(range(len(learning_rate_multipliers)), learning_rate_multipliers)
plt.imshow(np.mean(bleu4_array, axis = 1).reshape(1,-1))
plt.colorbar()
plt.show()

Na kraju koristimo kombinovani trening i validacioni skup da istreniramo konacan model za evaluaciju.

Izdvajamo mali deo ovog skupa (300 redova) za validaciju tokom treniranja, da bismo znali kad da prekinemo trening.

In [ ]:
df_train_val_np = df_train_val.to_numpy()
np.random.shuffle(df_train_val_np)
split_size = 300 
train_data = df_train_val_np[:train_size + val_size - split_size,]
val_data = df_train_val_np[train_size + val_size - split_size:,]
input_texts, target_texts = clean_texts(train_data[:,1], train_data[:,0])
input_word_index, target_word_index, max_input_seq_len, max_target_seq_len = analyse_texts(input_texts, target_texts)
input_pad_len = 80
target_pad_len = 60
num_input_words = len(input_word_index) - 1
num_target_words = len(target_word_index) - 1
inverted_input_word_index = {value: key for key,value in input_word_index.items()}
inverted_target_word_index = {value: key for (key,value) in target_word_index.items()}
input_embedding_matrix, target_embedding_matrix = load_embedding_data_get_matrices(inverted_input_word_index, inverted_target_word_index)
encoder_input_data, decoder_input_data, decoder_output_data = create_model_data(input_texts, target_texts, input_word_index, target_word_index, input_pad_len, target_pad_len)

input_texts_val, target_texts_val = clean_texts(val_data[:,1], val_data[:,0])
encoder_input_data_val, decoder_input_data_val, decoder_output_data_val = create_model_data(input_texts_val, target_texts_val, input_word_index, target_word_index, input_pad_len, target_pad_len)

input_texts_test, target_texts_test = clean_texts_df(df_test)
encoder_input_data_test, decoder_input_data_test, decoder_output_data_test = create_model_data(input_texts_test, target_texts_test, input_word_index, target_word_index, input_pad_len, target_pad_len)

In [ ]:
model_for_evaluation = GRU_Translation_Model(num_input_words, num_target_words, input_embedding_matrix, target_embedding_matrix)

other_layers = model_for_evaluation.layers[0].layers + model_for_evaluation.layers[1].layers 
embedding_layers = [other_layers.pop(2), other_layers.pop(-9)] 
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers = [(Adam(learning_rate), other_layers), (Adam(best_multiplier*learning_rate), embedding_layers)])
model_for_evaluation.compile(optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

early_stopping_safe = EarlyStopping(patience = 20, restore_best_weights = True, monitor = 'val_loss', mode = 'min', verbose = 1)
history = model_for_evaluation.fit([encoder_input_data, decoder_input_data], decoder_output_data, validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_output_data_val), epochs = 200, batch_size = 128, verbose = 1, callbacks = [early_stopping_safe])

In [ ]:
model_for_evaluation.summary(expand_nested = True)

In [ ]:
epoch_counter = range(len(history.history['loss']))
fig, (ax1, ax2) = plt.subplots(2,1)
ax1.plot(epoch_counter, history.history['loss'], label = 'Train Loss')
ax1.plot(epoch_counter, history.history['val_loss'], label = 'Validation Loss', linestyle = 'dashed')
ax1.legend()
ax2.plot(epoch_counter, history.history['acc'], label = 'Train accuracy')
ax2.plot(epoch_counter, history.history['val_acc'], label = 'Validation Accuracy', linestyle = 'dashed')
ax2.legend()

In [ ]:
wer, smooth_bleu4, smooth_bleu3, smooth_bleu2, smooth_bleu1 = evaluate(model_for_evaluation, input_texts_test, target_texts_test, input_word_index, target_word_index, inverted_target_word_index, input_pad_len, target_pad_len)
print('Results on test data:')
print('Word Error Rate: ', 100*wer) 
print('BLEU4(smooth): ', 100*smooth_bleu4) 
print('BLEU3(smooth): ', 100*smooth_bleu3) 
print('BLEU2(smooth): ', 100*smooth_bleu2) 
print('BLEU1(smooth): ', 100*smooth_bleu1) 